In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [4]:
### Loop the data lines
with open("../user_data_csv/recommendation_readme.csv", 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]

### Read csv
df = pd.read_csv("../user_data_csv/recommendation_readme.csv", header=None, delimiter=",", names=column_names)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,3493,3494,3495,3496,3497,3498,3499,3500,3501,3502
0,bmccann,['party_predictor \n A repo for a CS 229 final...,'pytorch-xla-transformer-language-model \n Th...,and is not intended to be used for training a...,this code triggered recompilation far too often,but these issues have now been resolved. \n ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fmassa,['torch-nn \n Additional functionnalities for ...,'imagine-nn \n Universite Paris-Est Marne-la-...,kH,dW,dH)\ninn.SpatialAveragePooling(kW,kH,dW,dH)\ninn.SpatialCrossResponseNormalization(size,[alpha = 0.0001],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,edouardelasalles,['Spatio-Temporal Neural Networks for Space-Ti...,and the 41 columns are the 41 space points.\n...,where A(i,j) = 1 means that series i is a direct nei...,and is 0 otherwise.','Dynamic Neural Language Models \n Code and s...,'Dynamic Author Representation \n Official im...,Sylvain Lamprier,Ludovic Denoyer \n IEEE International Confere...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,edouardoyallon,"[""ScatNetLight v0.1 - June 2015 \n ScatNetLigh...",Edouard Oyallon,Stéphane Mallat,accepted to CVPR 2015,http://arxiv.org/abs/1412.8659 \n Author cont...,edouard.oyallon@ens.fr\nContributor: Matthew ...,matthew.hirn@ens.fr \n Install ScatNetLight +...,'create_config_caltech_256','create_config_cifar_10' and 'create_config_c...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SeanNaren,"[""deepspeech.torch \n \n \n Implementation of ...",trained with the CTC activation function. \n ...,Master and Phoneme:\n* Master: This branch tr...,as well as a prediction script.\nThis branch ...,you can start training from these nets by usi...,Yuan Yang and Yan Xia for their significant c...,multi-gpu support and many other important fe...,and the encouraging words and support given t...,"""QlearningExample.torch \n \n Torch plays cat...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
COLUMN_NAMES=['USERNAME','README']
df_username_readme = pd.DataFrame(columns=COLUMN_NAMES)
df_username_readme.columns


TypeError: 'Index' object is not callable

In [89]:
df_tfidf = df[['bio']]
df_tfidf['id'] = df_tfidf.index
df_tfidf = df_tfidf[['id','bio']]
df_tfidf.head()

/var/folders/ml/8_bkc7k91cb777tm6mxkvfbc0000gn/T/ipykernel_42112/525773181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tfidf['id'] = df_tfidf.index


,id,bio
0,0,NaN
1,1,Research Scientist
2,2,NaN
3,3,NaN
4,4,NaN


In [90]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_tfidf['bio'].values.astype('U'))
tfidf_matrix.shape

(300, 1542)

In [91]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}

In [99]:
for idx, row in df_tfidf.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], df_tfidf['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]

print('done!')

done!


In [100]:
def item(id):
    return df_tfidf.loc[df_tfidf['id'] == id]['bio'].tolist()[0].split(' - ')[0]

In [101]:
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

recommend(item_id=11, num=5)

Recommending 5 products similar to Researcher @NVIDIA, Formerly Mila | CMU | MSR | FAIR | Element AI...
-------
Recommended: Research Scientist at FAIR
 (score:0.08801818621803285)
Recommended: Research Scientist in Meta AI Research (FAIR) (score:0.07802597074875811)
Recommended: Researcher Machine Learning and Vision at Concordia U/MILA/UdeM (score:0.07555146609620625)
Recommended: @scoutapp-ai  (score:0.06275448039732234)
Recommended: covariant.ai (score:0.06275448039732234)
